In [1]:
##Ignore all the warning message
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
from tqdm.auto import tqdm ### tqdm provdes a progress bar for computating a task
import spacy
from spacy.lang.en.stop_words import STOP_WORDS as stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np

In [2]:
##Read the dataset

git_root = "https://github.com/blueprints-for-text-analytics-python/blueprints-text/raw/master"
file = f"{git_root}/data/un-general-debates/un-general-debates-blueprint.csv.gz"

df = pd.read_csv(file)


,session,year,country,country_name,speaker,position,text
0,25,1970,ALB,Albania,Mr. NAS,NaN,33: May I first convey to our President the co...
1,25,1970,ARG,Argentina,Mr. DE PABLO PARDO,NaN,177.\t : It is a fortunate coincidence that pr...
2,25,1970,AUS,Australia,Mr. McMAHON,NaN,100.\t It is a pleasure for me to extend to y...
3,25,1970,AUT,Austria,Mr. KIRCHSCHLAEGER,NaN,155.\t May I begin by expressing to Ambassado...
4,25,1970,BEL,Belgium,Mr. HARMEL,NaN,"176. No doubt each of us, before coming up to ..."
...,...,...,...,...,...,...,...
7502,70,2015,YEM,Yemen,Mr. Abdrabuh Mansour Hadi Mansour,President,On behalf of the people and the Government of ...
7503,70,2015,YUG,Yugoslavia,Mr. Tomislav Nikolić,President,\nSeventy years have passed since the establis...
7504,70,2015,ZAF,South Africa,Jacob Zuma,President,I should like to congratulate the President an...
7505,70,2015,ZMB,Zambia,Mr. Edgar Chagwa Lungu,President,I would like to begin by thanking the Secretar...


In [17]:
pd.options.display.max_colwidth = 100 ### set display width
pd.set_option("display.max_colwidth", -1) ### print full content
df = df[['text']].dropna

In [4]:
"""
Clean the data and generate, for each article, lemmas, nounds, and noun-phrases.
"""
##clean the text 

def clean(text):
    # tags like <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text)
    # standalone sequences of hyphens like --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    # removing mentions 
    text = re.sub("@\S+", "", text)
    text = re.sub('[-%!@#$]', '', text)
    text = re.sub("@[A-Za-z0-9]+","",text)
    #Removing numerical data
    text = re.sub(r'\d+','',text)
    #Removing currencies 
    text = re.sub(r'[\$\d+\d+\$]','',text)
    #Handling all date formats
    text = re.sub(r'\d+[\.\/-]\d+[\.\/-]\d+', '', text)
    #Removing a hyperlink
    text = re.sub(r'https?:\/\/.*[\r\n]*', '',text)
    #Removing all punctuation
    text = re.sub(r'[^a-z0-9A-Z_]',' ',text)
    return text.strip()

df['text'] = df['text'].apply(clean)

In [5]:
nlp = spacy.load("en_core_web_sm")

In [7]:
 for i, row in tqdm(df.iterrows(), total=len(df)):
    doc = nlp(str(row["text"])) ### tokenization
    df.at[i, "lemmas"] = " ".join([token.lemma_ for token in doc]) ### generate lemmas for tokens

  0%|          | 0/7507 [00:00<?, ?it/s]

In [8]:
df

text  \
0     May I first convey to our President the congratulations of the Albanian delegation on his election to the Presidency of the twentyfifth session of the General Assembly    In taking up the work on the agenda of the twenty fifth session of the General Assembly  which is being held on the eve of the twentyfifth anniversary of the coming into force of the Charter of the United Nations  the peaceloving Member States would have wished to be in a position to present on this occasion some picture of positive and satisfactory activity on the part of the United Nations  The Albanian delegation  for its part  would have taken great pleasure in drawing up such a balance sheet of activities covering a quarter of a century  which is certainly no short period in the life of an international organization  Unfortunately  this is not the situation  Created on the day after victory had been achieved over the Powers of the Rome BerlinTokyo Axis and conceived in the spirit of the principles which predominated in the war of the antiFascist coalition  the Organization awakened in the whole of progressive humanity the hope that it would serve as an important factor in creating better international conditions in order to favor the cause of freedom  peace and world security  But its activities and a number of events which occurred in the world arena during that period disappointed the hopes of peoples  The United Nations  far from having contributed  as required by the fundamental provisions of the Charter  to the consolidation of international peace and security and the liberation struggle of peoples  has on the contrary  under the diktat of the imperialist powers  the United States of America foremost among them  embarked on a path diametrically opposed to those principles  and has served as an instrument favoring their policy of pillage  oppression and aggression against peaceloving peoples and countries    The utilization of the United Nations to serve this policy has resulted in the Organization not only not having been able to stay the hand of those committing aggression in many parts of the world  but having frequently helped the aggressors  its activity in this direction going so far  moreover  that under the flag of the United Nations there have been perpetrated some of the most barbarous aggressions experienced since the end of the Second World War  such as  among others  the armed aggressions in Korea and in the Congo    The whole of progressive mankind recalls with admiration the heroic struggle which  under the direction of the great Stalin  the Soviet Union waged  in cooperation with the AngloAmerican coalition  to defend the peaceful peoples and countries against Hitlerite Germany  The historic and decisive victories of the Red Army over the Nazi hordes and the great role played by the Soviet Union during the Second World War aroused great enthusiasm among peoples and awakened in them the hope for a better future  They followed with great interest and with particular joy the unflinching struggle which was waged by the Leninist Soviet Union in the world arena  as well as by its authentic representatives here in this Organization  in order to defend their vital interests and to sustain their legitimate struggle for liberation from the imperialist  colonialist and racist yoke  But following the usurpation of power in that country by the Khrushchev clique  the situation changed completely  and the struggle of the Soviet Union against American imperialism and its allies was replaced by the holy alliance between the Soviet Union and the United States against the peaceloving and freedomloving peoples and countries  It is thus that  so far as the United Nations is concerned  it has become a center of bargaining  maneuvering and underhand manipulations on the part of the American imperialists and the Soviet socialimperialists  designed to lull the vigilance of peoples and to disguise as best they can their policy of world hegemony  

In [9]:
# ##Clean lemmas
df['lemmas'] = df['lemmas'].apply(clean)

In [13]:
###Create noun verb pairs

noun_verb_pairs = []

speech = df['lemmas']
doc = nlp(str(speech)) ### tokenization

for i,token in enumerate(doc):
    if token.pos_ not in ('NOUN','PROPN'):
        continue
    for j in range(i+1,len(doc)):
        if doc[j].pos_ == 'VERB':
            noun_verb_pairs.append((token,doc[j]))
            break
print(noun_verb_pairs)

[(President, take), (congratulation, take), (delegation, take), (election, take), (Presidency, take), (session, take), (General, take), (Assembly, take), (work, wish), (agenda, wish), (session, wish), (General, wish), (Assembly, wish), (hold, wish), (eve, wish), (anniversary, wish), (come, wish), (force, wish), (Charter, wish), (United, wish), (Nations, wish), (peacelove, wish), (Member, wish), (States, wish), (position, present), (occasion, take), (picture, take), (activity, take), (part, take), (United, take), (Nations, take), (delegation, take), (part, take), (pleasure, draw), (balance, cover), (sheet, cover), (activity, cover), (quarter, be), (century, be), (period, be), (life, be), (organization, be), (situation, create), (day, achieve), (victory, achieve), (Powers, conceive), (Rome, conceive), (BerlinTokyo, conceive), (Axis, conceive), (spirit, predominate), (principle, predominate), (war, awaken), (antifascist, awaken), (coalition, awaken), (Organization, awaken), (whole, serve)

In [21]:
doc1 = nlp(str(noun_verb_pairs)) ### tokenization
text = []
for sent in doc1.sents: ### segment sentences
    text.append(sent)
text

[[(President, take), (congratulation, take), (delegation, take), (election, take), (Presidency, take), (session, take), (General, take), (Assembly, take), (work, wish), (agenda, wish), (session, wish), (General, wish), (Assembly, wish), (hold, wish), (eve, wish), (anniversary, wish), (come, wish), (force, wish), (Charter, wish), (United, wish), (Nations, wish), (peacelove, wish), (Member, wish), (States, wish), (position, present), (occasion, take), (picture, take), (activity, take), (part, take), (United, take), (Nations, take), (delegation, take), (part, take), (pleasure, draw), (balance, cover), (sheet, cover), (activity, cover), (quarter, be), (century, be), (period, be), (life, be), (organization, be), (situation, create), (day, achieve), (victory, achieve), (Powers, conceive), (Rome, conceive), (BerlinTokyo, conceive), (Axis, conceive), (spirit, predominate), (principle, predominate), (war, awaken), (antifascist, awaken), (coalition, awaken), (Organization, awaken), (whole, serve